<a href="https://colab.research.google.com/github/SaeidDaliriSusefi/SMAP-Downscalling/blob/main/SMAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import geemap

In [ ]:
ee.Authenticate()
ee.Initialize(project="ee-saeiddalirisu", opt_url='https://earthengine-highvolume.googleapis.com')

In [ ]:
map=geemap.Map(basemap="SATELLITE")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
roi = map.draw_last_feature.geometry()

In [78]:
time_start = '2020-01'
time_end   = '2020-04'

In [79]:
time_start = ee.Date(time_start)
time_end = ee.Date(time_end)

# Get the SMAP Soil Moisture image collection, select the 'soil_moisture_am' band, and filter by date range.
sm = ee.ImageCollection("NASA/SMAP/SPL3SMP_E/006") \
    .select('soil_moisture_am') \
    .filterDate(time_start, time_end)

# Create 10-day interval dates dynamically from time_start
list_10days = ee.List.sequence(0, time_end.difference(time_start, 'day'), 10) \
    .map(lambda i: time_start.advance(i, 'day'))
list_10days

In [80]:
# Define the mapping function in Python
def map_to_10day_image(start_date):
    start_date = ee.Date(start_date)
    end_date = start_date.advance(10, 'day')
    img = sm.filterDate(start_date, end_date).median()
    resample = img.resample().reproject(crs=img.projection().crs(), scale=1000)
    return resample.multiply(100).toInt() \
        .set('system:time_start', start_date.millis()) \
        .set('system:time_end', end_date.millis()) \
        .set('system:index', start_date.format('YYYY-MM-dd'))

# Apply the map function to the list of dates and convert to ImageCollection
sm_10days = ee.ImageCollection(list_10days.map(map_to_10day_image))
sm_10days


In [81]:
temp = ee.ImageCollection("MODIS/061/MOD11A1") \
    .select('LST_.*') \
    .filterDate(time_start, time_end)

# Function to compute 10-day composites
def map_to_temp_10days(start_date):
    start_date = ee.Date(start_date)
    end_date = start_date.advance(10, 'day')
    img = temp.filterDate(start_date, end_date).mean()
    lst_day = img.select('LST_Day_1km').multiply(0.02)
    lst_night = img.select('LST_Night_1km').multiply(0.02)
    lst_delta = lst_day.subtract(lst_night).rename('LST_delta')
    return lst_day.addBands(lst_delta) \
        .set('system:time_start', start_date.millis()) \
        .set('system:time_end', end_date.millis()) \
        .set('system:index', start_date.format('YYYY-MM-dd'))

# Apply function and convert to ImageCollection
temp_10days = ee.ImageCollection(list_10days.map(map_to_temp_10days))

temp_10days

In [82]:
sr = ee.ImageCollection("MODIS/061/MOD09GA") \
    .select('sur.*') \
    .combine(temp.select('LST_Day.*')) \
    .filterDate(time_start, time_end) \
    .map(lambda img: (
        # Mask surface reflectance with LST data and scale
        img.select('sur.*')
           .updateMask(img.select('LST.*'))
           .multiply(0.0001)
           .select(['sur_refl_b01', 'sur_refl_b02', 'sur_refl_b05', 'sur_refl_b06', 'sur_refl_b07'])  # For safety
           .addBands(
               img.select('sur.*')
                  .updateMask(img.select('LST.*'))
                  .multiply(0.0001)
                  .normalizedDifference(['sur_refl_b02', 'sur_refl_b01'])
                  .rename('ndvi')
           )
           .copyProperties(img, img.propertyNames())
    ))

# Create 10-day composite image collection
def map_to_sr_10days(start_date):
    start_date = ee.Date(start_date)
    end_date = start_date.advance(10, 'day')
    ndvi = sr.select('ndvi').filterDate(start_date, end_date).max()
    ms = sr.select('sur.*').filterDate(start_date, end_date).median()
    return ms.addBands(ndvi) \
        .set('system:time_start', start_date.millis()) \
        .set('system:time_end', end_date.millis()) \
        .set('system:index', start_date.format('YYYY-MM-dd'))

sr_10days = ee.ImageCollection(list_10days.map(map_to_sr_10days))
sr_10days

In [83]:
# Load MODIS land cover image (mode to get most frequent class)
landcover = ee.ImageCollection("MODIS/061/MCD12Q1").mode().select('LC_Type1')


In [84]:
# Combine sr_10days, temp_10days, and sm_10days
collection = sr_10days.combine(temp_10days).combine(sm_10days) \
    .map(lambda img: img.addBands(landcover).copyProperties(img, img.propertyNames()))

collection

In [85]:
def map_to_sm_1km(img):
    # Select training samples
    training_data = img.stratifiedSample(
        numPoints=10,
        classBand='soil_moisture_am',
        region=roi,
        scale=1000
    )

    # Train gradient boosted tree regressor
    model = ee.Classifier.smileGradientTreeBoost(80).train(
        features=training_data,
        classProperty='soil_moisture_am',
        inputProperties=img.bandNames()
    ).setOutputMode('REGRESSION')

    # Remove the target band from input
    band_names = img.bandNames().remove('soil_moisture_am')

    # Classify to predict soil moisture at 1km
    result = img.select(band_names).classify(model) \
        .rename('SM_1km') \
        .divide(100.0) \
        .toFloat()

    # Add original 9km soil moisture as 'SM_9km' and return the result
    return result.addBands(img.select(['soil_moisture_am'], ['SM_9km'])) \
        .copyProperties(img, ['system:time_start', 'system:time_end'])

# Apply function across all images in the collection
sm_1km = collection.map(map_to_sm_1km)
sm_1km


In [87]:
count = (sm_1km).size().getInfo()
image_list = sm_1km.select('SM_1km').toList(count)
image_list

In [92]:
Map = geemap.Map(basemap="SATELLITE")
for i in range(count):
    img = ee.Image(image_list.get(i)).clip(roi)

    date = img.get('system:index').getInfo()

    vmin = img.reduceRegion(
        reducer=ee.Reducer.min(),
        geometry=roi,
        bestEffort=True
    ).get('SM_1km').getInfo()
    vmax = img.reduceRegion(
        reducer=ee.Reducer.max(),
        geometry=roi,
        bestEffort=True
    ).get('SM_1km').getInfo()

    vis = {
        'min': vmin,
        'max': vmax,
        'palette': ['black', 'white']
    }

    Map.addLayer(img, vis, date)

Map.centerObject(roi, 6,)
Map


Map(center=[36.54456741102713, 47.009350999999974], controls=(WidgetControl(options=['position', 'transparent_…